In [1]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [3]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

from collections import Counter
label_count_dict = dict(Counter(train_y_list).most_common())
print label_count_dict

{0: 881, 1: 2093, 2: 5590, 3: 674, 4: 721}


In [4]:
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')

In [5]:
# z-score normalization
# scaler = preprocessing.StandardScaler().fit(train_x_list)
# train_x_list = scaler.transform(train_x_list)
# test_x_list = scaler.transform(test_x_list)

In [6]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [7]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [8]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [9]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

(9959,) (9959, 384)
(8257,) (8257, 384)


In [10]:
print train_x_list[0][:10]
print test_x_list[0][:10]

[  5.66733100e-03   3.11773100e-04   5.35555800e-03   5.30000000e+01
   9.60000000e+01   1.30774500e-03   4.18643100e-06   1.09633100e-03
   1.80239600e-06   1.34818000e-03]
[  1.23016500e-02   9.11361400e-05   1.22105100e-02   5.20000000e+01
   2.40000000e+01   1.31725500e-03   3.23349300e-07   1.29866300e-03
   6.89166500e-06   2.62522000e-03]


In [11]:
X = train_x_list
y = train_y_list

In [12]:
# record all predict_y
predict_y_lists = []

In [13]:
from hmmlearn.hmm import GMMHMM
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list)):
    train_x_lists[int(train_y_list[i])].append(train_x_list[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

# train models and get scores
models = []
scores = []
for i in range(5):
    X = np.array(train_x_lists[i])
    model = GMMHMM(n_mix=6, n_components=1).fit(X)
    models.append(model)

    score = []
    for XX in test_x_list:
        score.append(model.score(np.array(XX)))
    scores.append(score)

predict_y_list = np.argmax(np.array(scores).T, axis = 1)
calc_score(test_y_list, predict_y_list.tolist())

number of class 0 : 881
number of class 1 : 2093
number of class 2 : 5590
number of class 3 : 674
number of class 4 : 721


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/hmmlearn/hmm.py:593: RuntimeWarning: divide by zero encountered in log
  lgmm_posteriors = (np.log(g.predict_proba(X))


Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.30919651271699855, 0.31874470182958359, 0.31065820289388463, None)
weighted:  (0.54635336739904294, 0.52537241128739254, 0.5330757944804223, None)


In [14]:
predict_y_lists.append(predict_y_list)

In [16]:
# Linear Discriminant Analysis
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
predict_y_list = lda.fit(train_x_list, train_y_list).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:20:28
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.35936656041590076, 0.3375721378731863, 0.34056468342636742, None)
weighted:  (0.58814106817086764, 0.63558193048322631, 0.60588664292089445, None)
end: 2016-12-27 13:20:29


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [17]:
predict_y_lists.append(predict_y_list)

In [18]:
# z-score normalization
scaler = preprocessing.StandardScaler().fit(train_x_list)
train_x_list = scaler.transform(train_x_list)
test_x_list = scaler.transform(test_x_list)

In [19]:
# 2-Layer MLP
batch_size = 500
train_iter = mx.io.NDArrayIter(train_x_list, train_y_list, batch_size=batch_size)
test_iter = mx.io.NDArrayIter(test_x_list, test_y_list, batch_size=batch_size)

data = mx.symbol.Variable('data')
# data = mx.symbol.Dropout(data=data, p=0.2)
# fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=2048)
# act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = data, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=20)
mlp  = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax')

# mx.viz.plot_network(mlp)

print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

model = mx.model.FeedForward(
    ctx = mx.cpu(),       # Run on CPU
    symbol = mlp,         # Use the network we just defined
    num_epoch = 100,       # Train for 10 epochs
    learning_rate = 0.1,  # Learning rate
    momentum = 0.9,       # Momentum for SGD with momentum
    wd = 0.00001)         # Weight decay for regularization
model.fit(
    X=train_iter,  # Training data set
    eval_data=test_iter)#,  # Testing data set. MXNet computes scores on test set every epoch
    #batch_end_callback = mx.callback.Speedometer(batch_size, 2000))  # Logging module to print out progress

print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

predict_y_score_list = model.predict(test_x_list)
predict_y_list = np.argmax(predict_y_score_list, axis=1)

calc_score(test_y_list, predict_y_list)

INFO:root:Start training with [cpu(0)]


start: 2016-12-27 13:20:29


INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.184
INFO:root:Epoch[0] Validation-accuracy=0.636118
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.199
INFO:root:Epoch[1] Validation-accuracy=0.567294
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=0.132
INFO:root:Epoch[2] Validation-accuracy=0.569176
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=0.180
INFO:root:Epoch[3] Validation-accuracy=0.542824
INFO:root:Epoch[4] Resetting Data Iterator
INFO:root:Epoch[4] Time cost=0.187
INFO:root:Epoch[4] Validation-accuracy=0.629059
INFO:root:Epoch[5] Resetting Data Iterator
INFO:root:Epoch[5] Time cost=0.218
INFO:root:Epoch[5] Validation-accuracy=0.614118
INFO:root:Epoch[6] Resetting Data Iterator
INFO:root:Epoch[6] Time cost=0.124
INFO:root:Epoch[6] Validation-accuracy=0.631412
INFO:root:Epoch[7] Resetting Data Iterator
INFO:root:Epoch[7] Time cost=0.130
INFO:root:Epoch[7] Validation-accuracy=0

start: 2016-12-27 13:20:52
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.34635143803308399, 0.33137320813812043, 0.33176352121039965, None)
weighted:  (0.57323056405785311, 0.60227685600096892, 0.58407894981417363, None)


In [20]:
predict_y_lists.append(predict_y_list)

In [21]:
# SVM with class_weight='balanced'
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

clf = svm.SVC(class_weight='balanced').fit(train_x_list, train_y_list)
print clf
predict_y_list = clf.predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:20:52
SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.33471047772992807, 0.41088171030458698, 0.35022002324791102, None)
weighted:  (0.60335579154366337, 0.5066004602155747, 0.5389480771387325, None)
end: 2016-12-27 13:22:19


In [22]:
predict_y_lists.append(predict_y_list)

In [23]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
clf = AdaBoostClassifier()
predict_y_list = clf.fit(train_x_list, train_y_list).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:22:19
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.30088067856482786, 0.31591976743535544, 0.30531255304262173, None)
weighted:  (0.55458441677032522, 0.60627346493883982, 0.57650532167728252, None)
end: 2016-12-27 13:22:39


In [24]:
predict_y_lists.append(predict_y_list)

In [25]:
# SMOTE up-sampling training data
sm = SMOTE(kind='regular')
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list, train_y_list)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list_smote)):
    train_x_lists[int(train_y_list_smote[i])].append(train_x_list_smote[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 1: 2093, 0: 881, 4: 721, 3: 674})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4916 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 3: 5590, 1: 2093, 0: 881, 4: 721})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4869 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({2: 5590, 3: 5590, 4: 5590, 1: 2093, 0: 881})
DEBUG:imblearn.base:Finding the 5 nearest neighbours ...
DEBUG:imblearn.base:Create synthetic samples ...
INFO:imblearn.base:Generated 4709 new samples ...
INFO:imblearn.base:Compute classes statistics ...
INFO:imblearn.base:5 classes detected: Counter({0: 5590,

number of class 0 : 5590
number of class 1 : 5590
number of class 2 : 5590
number of class 3 : 5590
number of class 4 : 5590


In [26]:
full_predict_y_lists = predict_y_lists

In [50]:
predict_y_lists = []
predict_y_lists.append(full_predict_y_lists[0])
predict_y_lists.append(full_predict_y_lists[1])
predict_y_lists.append(full_predict_y_lists[2])
predict_y_lists.append(full_predict_y_lists[3])
# predict_y_lists.append(full_predict_y_lists[4])

In [51]:
# FUSION
from collections import Counter

fusion_predict_y_list = []
for i in xrange(len(predict_y_list)):
    c = Counter([predict_y_lists[j][i] for j in xrange(len(predict_y_lists))])
    most_count = c.most_common(1)[0][1]
    most_label = [tup[0] for tup in c.most_common() if tup[1] == most_count]
    most_label_dict = {key: label_count_dict[key] for key in most_label}
    
    # better accuracy
#     fusion_predict_y_list.append(max(most_label_dict, key=most_label_dict.get))
    # better recall
    fusion_predict_y_list.append(min(most_label_dict, key=most_label_dict.get))

calc_score(test_y_list, fusion_predict_y_list)

Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.36571270204469675, 0.38503110734573298, 0.36741045991337756, None)
weighted:  (0.59641959260844024, 0.59779580961608336, 0.59507108216534987, None)


In [40]:
for i in range(len(predict_y_lists)):
    print predict_y_lists[i][:30].astype(int)
print '--------------------'
print test_y_list[:30]
print '--------------------'
print np.array(fusion_predict_y_list[:30])

[2 4 2 4 2 2 0 2 4 0 3 2 2 2 2 2 2 2 2 2 2 4 0 2 4 2 3 2 2 2]
[2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 0 1 2 2 2 2 2 2]
[2 2 2 2 2 0 2 2 2 2 3 2 3 3 2 1 1 4 2 2 2 2 4 1 2 2 4 2 4 1]
[2 2 2 4 2 0 4 2 1 0 3 2 4 2 1 1 1 4 3 2 2 2 0 1 2 4 4 2 4 4]
--------------------
[2 2 2 2 2 4 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2]
--------------------
[2 2 2 4 2 0 2 2 2 0 3 2 2 2 2 1 1 4 2 2 2 2 0 1 2 2 4 2 4 2]
